In [1]:
!pip install --upgrade --pre pythainlp
!pip install pyLDAvis

In [88]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [89]:
df=pd.read_csv('/Users/nunna/iCloudDrive/MADT/เทอม3_2565/MADT8101/Homework/6 VOC/TripadvisorReview_SalaSamui.csv')

In [90]:
df.tail()

,ReviewID,Review
8,9,ถ่ายรูปสวยทุกมุม และถ่ายรูปสวยทุกมุม
9,10,พนักงานดูแลดี ใส่ใจรายละเอียด เรือคยัค และ Pa...
10,11,ที่พักสวยมาก พนักงานบริการดี อาหารอร่อยค่ะ
11,12,มุมถ่ายรูปสวย ห้องก็สวยงาม พนักงานบริการดี
12,13,น้ำทะเลใส หาดทรายขาวละเอียด พนักงานบริการดี


In [358]:
#Tokenize Words
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['',' ','  ','\n','ค่ะ','คะ','โรงแรม','นะคะ','ๆ','('')','(',')','(' ,')','(' , ')','(', ')','ๆ','มาก','บริการ','ดีมาก','อย่างมาก','เลย','แนะนำ','ไม่มีที่ติ','เหมาะกับ','มมม','ความ','แยก','ไร้','หาย','ติ','เรื่อง','ให้การ','เข้าหา','ริม','ดี','น่ารัก','ยอดเยี่ยม','สุดยอด','ดีงาม','อัธ', 'เยี่ยม', 'ขนาด','และ','พาย','สวย','สวยงาม','มากๆ','สาย','ตกลง','ตก','มุม','มอ', 'มินิ', 'ล','เเต่ง','ใส','ละเอียด','หลากหลาย', 'อา', 'อาด', 'ใจ', 'ใส่', 'เหมาะ', 'รายละเอียด', 'สวยสะ', 'มื้อ','สี','ขาว']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize(str(sentence), engine='longest')
    for word in words:
        if word not in screening_words:
            merged = merged + ',' + word
    return merged[1:]

In [359]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [360]:
df.tail()

,ReviewID,Review,Review_tokenized,topics,score
8,9,ถ่ายรูปสวยทุกมุม และถ่ายรูปสวยทุกมุม,"ถ่ายรูป,ถ่ายรูป",0,0.067478
9,10,พนักงานดูแลดี ใส่ใจรายละเอียด เรือคยัค และ Pa...,"พนักงาน,ดูแล,ใส่ใจ,เรือ,คยัค,paddleboard",0,0.944313
10,11,ที่พักสวยมาก พนักงานบริการดี อาหารอร่อยค่ะ,"ที่พัก,พนักงานบริการ,อาหาร,อร่อย",0,0.919606
11,12,มุมถ่ายรูปสวย ห้องก็สวยงาม พนักงานบริการดี,"ถ่ายรูป,ห้อง,พนักงานบริการ",0,0.048933
12,13,น้ำทะเลใส หาดทรายขาวละเอียด พนักงานบริการดี,"น้ำทะเล,หาดทราย,พนักงานบริการ",0,0.920409


In [361]:
#Create Dictionary
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [362]:
print(dictionary.token2id.keys())

dict_keys(['paddleboard', 'คายัค', 'ดีไซน์', 'ตัวอาคาร', 'พนักงาน', 'ห้อง', 'เตียง', 'เรือ', 'เส้นโค้ง', 'แปลกตา', 'ได้รูป', 'การถ่ายรูป', 'ทะเล', 'ที่พัก', 'ริมหาด', 'ถ่ายรูป', 'ห้องพัก', 'ตกแต่ง', 'บรรยากาศ', 'ดูแล', 'สะอาด', 'อร่อย', 'อาหาร', 'กิจกรรม', 'สระ', 'paddlebord', 'พระอาทิตย์', 'วิว', 'อาหารเช้า', 'เครื่องดื่ม', 'โชว์', 'ไฟ', 'คยัค', 'ใส่ใจ', 'พนักงานบริการ', 'น้ำทะเล', 'หาดทราย'])


In [363]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [635]:
#Topic Modeling
num_topics = 3
chunksize = 2000 # size of the doc looked at every pass
passes = 50 # number of passes through documents
iterations = 50
eval_every =1 # Don't evaluate model perplexity, takes too much time

#make a index to word dictionary
temp = dictionary[0] #This is only to load the dictionary
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                                    alpha='auto', eta='auto', \
                                    iterations=iterations, num_topics=num_topics, \
                                    passes=passes, eval_every=eval_every)

CPU times: total: 125 ms
Wall time: 152 ms


In [636]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.036140  0.085114       1        1  41.124325
1     -0.106086 -0.016345       2        1  39.641484
0      0.069946 -0.068769       3        1  19.234190, topic_info=             Term      Freq     Total Category  logprob  loglift
15        ถ่ายรูป  4.000000  4.000000  Default   30.000  30.0000
4         พนักงาน  3.000000  3.000000  Default   29.000  29.0000
16        ห้องพัก  1.000000  1.000000  Default   28.000  28.0000
11     การถ่ายรูป  1.000000  1.000000  Default   27.000  27.0000
20          สะอาด  1.000000  1.000000  Default   26.000  26.0000
..            ...       ...       ...      ...      ...      ...
0     paddleboard  0.171877  3.397518   Topic3   -4.332  -1.3355
34  พนักงานบริการ  0.171877  2.696732   Topic3   -4.332  -1.1045
1           คายัค  0.171877  2.704180   Topic3   -4.332  -1.1073
5            ห้อง  0.171877  2.696733   Topic3   -4.332  -1.1045
2          ดีไซน์  0.171877  2.010841   Topic3   -4.332  -0.8110

[136 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0         1  0.588665    paddleboard
0         2  0.294333    paddleboard
25        2  0.759009     paddlebord
11        3  0.871651     การถ่ายรูป
23        2  0.998304        กิจกรรม
32        1  0.754743           คยัค
1         1  0.739596          คายัค
1         2  0.369798          คายัค
2         1  0.497304         ดีไซน์
2         2  0.497304         ดีไซน์
19        1  0.543306           ดูแล
19        3  0.543306           ดูแล
17        2  0.759009         ตกแต่ง
3         1  0.754743       ตัวอาคาร
15        2  0.843928        ถ่ายรูป
12        2  0.545513           ทะเล
12        3  0.545513           ทะเล
13        2  0.545513         ที่พัก
13        3  0.545513         ที่พัก
35        1  0.754743        น้ำทะเล
18        2  0.759009       บรรยากาศ
4         1  0.560875        พนักงาน
4         3  0.560875        พนักงาน
34        1  0.370819  พนักงานบริการ
34        2  0.741638  พนักงานบริการ
26        2  0.759009     พระอาทิตย์
14        1  0.543306         ริมหาด
14        3  0.543306         ริมหาด
27        2  0.759009            วิว
24        2  0.759009            สระ
20        3  0.871651          สะอาด
36        1  0.754743        หาดทราย
5         1  0.370819           ห้อง
5         2  0.741638           ห้อง
16        3  0.600985        ห้องพัก
21        2  0.545513          อร่อย
21        3  0.545513          อร่อย
22        2  0.545513          อาหาร
22        3  0.545513          อาหาร
28        1  0.754743      อาหารเช้า
29        1  0.754743    เครื่องดื่ม
6         1  0.754743          เตียง
7         1  0.489787           เรือ
7         2  0.489787           เรือ
8         1  0.754743       เส้นโค้ง
9         1  0.754743         แปลกตา
30        1  0.754743           โชว์
33        1  0.754743          ใส่ใจ
10        1  0.754743         ได้รูป
31        1  0.754743             ไฟ, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [637]:
model.show_topic(1)

[('ถ่ายรูป', 0.16006954),
 ('พนักงานบริการ', 0.059371267),
 ('เรือ', 0.05937088),
 ('ห้อง', 0.059370782),
 ('กิจกรรม', 0.059370685),
 ('ทะเล', 0.03392615),
 ('ที่พัก', 0.033926148),
 ('อาหาร', 0.033926137),
 ('อร่อย', 0.033926137),
 ('paddleboard', 0.03392613)]

In [638]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [639]:
df.head()

,ReviewID,Review,Review_tokenized,topics,score
0,1,ดีไซน์เป็นเส้นโค้งแปลกตา ตัวอาคารสีขาวมินิมอล ...,"ดีไซน์,เส้นโค้ง,แปลกตา,ตัวอาคาร,ได้รูป,พนักงาน...",2,0.988216
1,2,ที่พักสวย ใกล้ทะเล เหมาะแก่การถ่ายรูป พนักงานร...,"ที่พัก,ทะเล,การถ่ายรูป,พนักงาน,ริมหาด",0,0.972085
2,3,โรงแรมที่สวยมาก มุมถ่ายรูปเยอะ พนักงานอาใจใส่ด...,"ถ่ายรูป,พนักงาน,ห้องพัก",0,0.651104
3,4,บรรยากาศการตกแต่งดีรายละเอียดในดีไซน์ มุมถ่ายร...,"บรรยากาศ,ตกแต่ง,ดีไซน์,ถ่ายรูป",0,0.010115
4,5,ห้องพักสะอาด การตกเเต่งสวยงาม อาหารทุกมื้ออร่อ...,"ห้องพัก,สะอาด,อาหาร,อร่อย,พนักงาน,ดูแล",0,0.976597


In [640]:
df.to_excel('/Users/nunna/iCloudDrive/MADT/เทอม3_2565/MADT8101/Homework/6 VOC/ResultVOC_Samui.xlsx', index=False)